# Train and Evaluate EfficientNet


***This notebook serves as debuging file for model loading as at some point of time we encountered that there was an issue with the way saved|loaded weights.***

## Setup

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import plotly.graph_objs as go

from keras.models import load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import MobileNetV2, MobileNet, MobileNetV3Small, EfficientNetB0, EfficientNetB1, EfficientNetB3, EfficientNetB4, EfficientNetB7
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_curve, auc

import plotly.io as pio

Setup pipeline veriables

In [ ]:
SEED = 20
DRIVE_PATH = "/kaggle"
DATA_DIRECTORY = f"{DRIVE_PATH}/input/sneakers-recognition-dataset-v22/Sneakers_Recognition_DataSet"
SHOW_DATASET_STRUCTURE = True
SHOW_IMAGE_SHAPE_MEANS = True
ShOW_IMAGE_SHAPE_PLOT = True
DRAW_CLASS_DISTRIBUTION = True

# IMAGE_SIZE = (300, 300)

# IMAGE_SIZE = (244, 244)

# IMAGE_SIZE = (224, 224)
IMAGE_SIZE = (240, 240)
BATCH_SIZE = 32
EPOCHS = 13
NUMBER_LAYERS_TO_FREEZE = 0 
MODEL_CLASS_CALLBACK = EfficientNetB1
SHOW_NUMBER_OF_LAYERS_IN_BASE_MODEL = True
SHOW_MODEL_SUMMARY = False

MODEL_SAVING_PATH = f"{DRIVE_PATH}/working/models/DATA_V2_{MODEL_CLASS_CALLBACK}_{NUMBER_LAYERS_TO_FREEZE}Frozen_{BATCH_SIZE}Batch_{IMAGE_SIZE[0]}x{IMAGE_SIZE[1]}Size_AugV2"

In [ ]:
from numpy.random import seed 
seed(SEED) # keras seed fixing import

import tensorflow
tensorflow.random.set_seed(SEED) # tensorflow seed fixing

## Data insights

In [ ]:
def list_directories(root_dir):
    """
    Recursively lists all directories and subdirectories within the specified directory.

    Args:
    - root_dir (str): The directory to list.

    Returns:
    - directories (list): A list of directory paths.
    """
    directories = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        directories.append(dirpath)
        for dirname in dirnames:
            directories.append(os.path.join(dirpath, dirname))
    return directories


if SHOW_DATASET_STRUCTURE:
    directories_structure = list_directories(DATA_DIRECTORY)
    for directory in directories_structure:
        print(directory)


In [ ]:
# Function to calculate mean dimensions of images in a directory
def calculate_mean_dimensions(directory):
    total_width = 0
    total_height = 0
    total_images = 0

    # Iterate through files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"): # Add other formats if needed
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path)
            if img is not None:
                total_width += img.shape[1]
                total_height += img.shape[0]
                total_images += 1

    # Calculate mean dimensions
    if total_images > 0:
        mean_width = total_width / total_images
        mean_height = total_height / total_images
        return mean_width, mean_height
    else:
        return 0, 0

if SHOW_IMAGE_SHAPE_MEANS:
    # Iterate through subdirectories
    for subdir in os.listdir(DATA_DIRECTORY):
        subdir_path = os.path.join(DATA_DIRECTORY, subdir)
        if os.path.isdir(subdir_path):
            mean_width, mean_height = calculate_mean_dimensions(subdir_path)
            print(f"Directory: {subdir}, Mean Width: {mean_width}, Mean Height: {mean_height}")


In [ ]:
def collect_dimensions(directory):
    widths = []
    heights = []

    # Iterate through files in the directory
    for root, _, files in os.walk(directory):
        print(f"Gathering data from {root} directory")
        for filename in files:
            if filename.endswith(".jpg") or filename.endswith(".png"): # Add other formats if needed
                img_path = os.path.join(root, filename)
                img = cv2.imread(img_path)
                if img is not None:
                    widths.append(img.shape[1])
                    heights.append(img.shape[0])

    return widths, heights

def plot_dimension_distribution(widths, heights):
    # Create histograms
    width_hist = go.Histogram(x=widths, name='Width', marker_color='blue')
    height_hist = go.Histogram(x=heights, name='Height', marker_color='green')

    # Create figure
    fig = go.Figure(data=[width_hist, height_hist])

    # Update layout
    fig.update_layout(
        title='Image Dimension Distribution',
        xaxis=dict(title='Dimension'),
        yaxis=dict(title='Frequency'),
        barmode='overlay',
        bargap=0.1
    )

    # Show plot
    fig.show()

def visualize_distribution(directory):
    widths, heights = collect_dimensions(directory)
    plot_dimension_distribution(widths, heights)


if ShOW_IMAGE_SHAPE_PLOT:
  # Visualize distribution
  visualize_distribution(DATA_DIRECTORY)


In [ ]:
# Create train and validation data generators
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range = 30,
    shear_range = 0.2,
    height_shift_range = 0.2,
    width_shift_range = 0.2,
    brightness_range = (0.2, 1.0),
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

train_generator = train_datagen.flow_from_directory(
    DATA_DIRECTORY,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',  # specify this as training data
    seed=SEED,
)

validation_generator = train_datagen.flow_from_directory(
    DATA_DIRECTORY,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',  # specify this as validation data
    seed=SEED,
)

all_labels = set(train_generator.labels).union(validation_generator.labels)
number_of_labels = len(all_labels)

In [ ]:
validation_generator.class_indices

In [ ]:
def draw_class_distribution_bar_plot(class_distribution, save_path):
    sorted_class_distribution = dict(sorted(class_distribution.items()))
    fig = go.Figure(data=[go.Bar(x=list(sorted_class_distribution.keys()), y=list(sorted_class_distribution.values()))])
    fig.update_layout(title='Class Distribution',
                      xaxis_title='Class',
                      yaxis_title='Count',
                      width=1000,
                      height=600)
    fig.write_html(save_path)  # Save the plot to the specified path as HTML
    fig.show()

# Example usage of drawing class distribution bar plot
def draw_class_distribution(generator, save_path):
    class_indices_mapping = generator.class_indices

    # Reverse the mapping to get class names from indices
    class_names = {v: k for k, v in class_indices_mapping.items()}
    
    class_distribution = {}
    for label in generator.labels:
        if label in class_distribution:
            class_distribution[label] += 1
        else:
            class_distribution[label] = 1

    draw_class_distribution_bar_plot(class_distribution, save_path)
    

if DRAW_CLASS_DISTRIBUTION:
    draw_class_distribution(train_generator, os.path.join(f"{DRIVE_PATH}/working", "class_distribution_train.html"))
    draw_class_distribution(validation_generator, os.path.join(f"{DRIVE_PATH}/working", "class_distribution_validation.html"))

## Model training

In [ ]:
# Define the input layer with your desired input shape
input_layer = Input(shape=(*IMAGE_SIZE, 3))

# Load the EfficientNetB0 model with pre-trained weights, excluding the top layer
base_model = MODEL_CLASS_CALLBACK(input_tensor=input_layer, weights='imagenet', include_top=False)

In [ ]:
# Count the number of trainable and non-trainable layers
def count_trainable_layers(model):
    trainable_layers = 0
    non_trainable_layers = 0

    for layer in model.layers:
        if layer.trainable:
            trainable_layers += 1
        else:
            non_trainable_layers += 1

    return trainable_layers, non_trainable_layers

if SHOW_NUMBER_OF_LAYERS_IN_BASE_MODEL:
    # Get the number of trainable and non-trainable layers
    trainable_layers, non_trainable_layers = count_trainable_layers(base_model)

    print("Number of trainable layers:", trainable_layers)
    print("Number of non-trainable layers:", non_trainable_layers)


In [ ]:
if SHOW_MODEL_SUMMARY:
    base_model.summary()

In [ ]:
# number_of_labels = 33

In [ ]:
def create_model():
    base_model = MODEL_CLASS_CALLBACK(input_tensor=input_layer, weights='imagenet', include_top=False)
    # # Freeze layers
    for layer in base_model.layers[:NUMBER_LAYERS_TO_FREEZE]:
        layer.trainable = False
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(number_of_labels, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

model = create_model()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import LearningRateScheduler

# Define the learning rate schedule function
def lr_schedule(epoch):
    initial_lr = 0.001
    if epoch is set([15, 25]):
        return initial_lr * 0.1
    return initial_lr

# Define the LearningRateScheduler callback
lr_scheduler = LearningRateScheduler(lr_schedule)

In [ ]:
# Ensure the directory exists
os.makedirs(MODEL_SAVING_PATH, exist_ok=True)

# Define a callback to save the model weights after every epoch
checkpoint_filepath = MODEL_SAVING_PATH + "/model_weights_epoch_{epoch:02d}.weights.h5"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=False,
    verbose=1
)

Train our model

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, lr_scheduler]
)

In [ ]:
# Save the trained model
model.save(f"{MODEL_SAVING_PATH}/model.h5")

## Model evaluation

In [ ]:
# Get training and validation loss values
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Get training and validation accuracy values
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Create traces for loss
loss_trace = go.Scatter(x=list(range(1, len(train_loss) + 1)), y=train_loss, mode='lines', name='Training Loss')
val_loss_trace = go.Scatter(x=list(range(1, len(val_loss) + 1)), y=val_loss, mode='lines', name='Validation Loss')

# Create traces for accuracy
acc_trace = go.Scatter(x=list(range(1, len(train_acc) + 1)), y=train_acc, mode='lines', name='Training Accuracy')
val_acc_trace = go.Scatter(x=list(range(1, len(val_acc) + 1)), y=val_acc, mode='lines', name='Validation Accuracy')

# Create figure for loss
loss_fig = go.Figure(data=[loss_trace, val_loss_trace])
loss_fig.update_layout(title='Training and Validation Loss',
                       xaxis_title='Epoch',
                       yaxis_title='Loss')

# Create figure for accuracy
acc_fig = go.Figure(data=[acc_trace, val_acc_trace])
acc_fig.update_layout(title='Training and Validation Accuracy',
                      xaxis_title='Epoch',
                      yaxis_title='Accuracy')

# Save figures as HTML files
pio.write_html(loss_fig, f"{MODEL_SAVING_PATH}/loss_figure.html")
pio.write_html(acc_fig, f"{MODEL_SAVING_PATH}/accuracy_figure.html")


In [ ]:
def get_true_and_predicted(generator, model):
    # Get true labels and predicted labels
    true_labels = []
    predicted_labels = []
    predicted_probabilities = []
    for i in range(len(generator)):
        batch = generator[i]
        true_labels.extend(np.argmax(batch[1], axis=1))  # Extract true labels from the batch
        predictions = model.predict(batch[0])
        predicted_labels.extend(np.argmax(predictions, axis=1))  # Extract predicted labels
        predicted_probabilities.extend(predictions)  # Extract predicted probabilities
    return true_labels, predicted_labels, predicted_probabilities

In [ ]:
# Function to plot confusion matrix using Plotly
def plot_confusion_matrix(conf_matrix, class_names, title, save_path=None):
    trace = go.Heatmap(z=np.flipud(conf_matrix),  # Reverse the y-axis
                       x=class_names,
                       y=class_names[::-1],  # Reverse the y-axis labels
                       colorscale='Blues',
                       colorbar=dict(title='Count'),
                       )
    layout = go.Layout(title=title,
                       xaxis=dict(title='Predicted labels'),
                       yaxis=dict(title='True labels'),
                       width=1000,  # Specify width of the plot
                       height=1000,  # Specify height of the plot
#                        legend=dict(title='Legend'),  # Include legend
                       )
    fig = go.Figure(data=[trace], layout=layout)
    if save_path:
        fig.write_html(save_path)  # Save the plot to the specified path as HTML
    fig.show()

# Function to calculate metrics and plot ROC-AUC curve
def calculate_metrics_and_plot_roc(true_labels, predicted_probabilities, class_names, title, save_path=None):
    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predicted_probabilities.argmax(axis=1))

    # Calculate top-5 accuracy
    top5_accuracy = top_k_accuracy(true_labels, predicted_probabilities, k=5)

    # Calculate F1 score
    f1 = f1_score(true_labels, predicted_probabilities.argmax(axis=1), average='macro')

    # Calculate precision
    precision = precision_score(true_labels, predicted_probabilities.argmax(axis=1), average='macro')

    # Calculate recall
    recall = recall_score(true_labels, predicted_probabilities.argmax(axis=1), average='macro')

    # Plot ROC-AUC curve
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(len(class_names)):
        fpr[i], tpr[i], _ = roc_curve(true_labels == i, predicted_probabilities[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Plot ROC-AUC curve
    fig_roc = go.Figure()
    for i in range(len(class_names)):
        fig_roc.add_trace(go.Scatter(x=fpr[i], y=tpr[i], mode='lines', name=f'{class_names[i]} (AUC = {roc_auc[i]:0.2f})'))
    fig_roc.update_layout(title='ROC Curve',
                          xaxis_title='False Positive Rate',
                          yaxis_title='True Positive Rate',
                          width=1000,
                          height=1000)
    if save_path:
        fig_roc.write_html(save_path)  # Save the plot to the specified path as HTML
    fig_roc.show()

    print(f'Accuracy: {accuracy}')
    print(f'Top-5 Accuracy: {top5_accuracy}')
    print(f'F1 Score: {f1}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')

# Predict labels and generate confusion matrix
def plot_confusion_matrix_and_metrics(generator, model, class_names, title, save_path=None):
    # Get true labels and predicted labels
    true_labels, predicted_labels, predicted_probabilities = get_true_and_predicted(generator, model)
    # Generate confusion matrix
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    plot_confusion_matrix(conf_matrix, class_names, title, save_path)

    # Calculate metrics and plot ROC-AUC curve
    calculate_metrics_and_plot_roc(np.array(true_labels), np.array(predicted_probabilities), class_names, title, save_path=None)

# Function to calculate top-k accuracy
def top_k_accuracy(true_labels, predicted_probabilities, k=5):
    top_k_predictions = np.argsort(predicted_probabilities, axis=1)[:, -k:]  # Get top k predictions
    matches = np.zeros_like(true_labels)
    for i in range(k):
        matches += (top_k_predictions[:, i] == true_labels)
    top_k_accuracy = np.mean(matches)
    return top_k_accuracy


class_names = [k for k, v in dict(sorted(train_generator.class_indices.items(), key=lambda item: item[1])).items()]

# Call above plot and metrics functions
plot_confusion_matrix_and_metrics(train_generator, model, class_names, 'Confusion Matrix - Training Set', save_path=f"{MODEL_SAVING_PATH}/training_set_plotly.html")
plot_confusion_matrix_and_metrics(validation_generator, model, class_names, 'Confusion Matrix - Validation Set', save_path=f"{MODEL_SAVING_PATH}/validation_set_plotly.html")

In [ ]:
# Function to plot confusion matrix using Plotly
def plot_confusion_matrix(conf_matrix, class_names, title, save_path=None):
    trace = go.Heatmap(z=np.flipud(conf_matrix),  # Reverse the y-axis
                       x=class_names,
                       y=class_names[::-1],  # Reverse the y-axis labels
                       colorscale='Blues',
                       colorbar=dict(title='Count'),
                       )
    layout = go.Layout(title=title,
                       xaxis=dict(title='Predicted labels'),
                       yaxis=dict(title='True labels'),
                       width=500,  # Specify width of the plot
                       height=500,  # Specify height of the plot
#                        legend=dict(title='Legend'),  # Include legend
                       )
    fig = go.Figure(data=[trace], layout=layout)
    if save_path:
        fig.write_html(save_path)  # Save the plot to the specified path as HTML
    fig.show()

# Function to calculate metrics and plot ROC-AUC curve
def calculate_metrics_and_plot_roc(true_labels, predicted_probabilities, class_names, title, save_path=None):
    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predicted_probabilities.argmax(axis=1))

    # Calculate top-5 accuracy
    top5_accuracy = top_k_accuracy(true_labels, predicted_probabilities, k=5)

    # Calculate F1 score
    f1 = f1_score(true_labels, predicted_probabilities.argmax(axis=1), average='macro')

    # Calculate precision
    precision = precision_score(true_labels, predicted_probabilities.argmax(axis=1), average='macro')

    # Calculate recall
    recall = recall_score(true_labels, predicted_probabilities.argmax(axis=1), average='macro')

    # Plot ROC-AUC curve
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i, v in enumerate(class_names):
        fpr[i], tpr[i], _ = roc_curve(true_labels == v, predicted_probabilities[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Plot ROC-AUC curve
    fig_roc = go.Figure()
    for i in range(len(class_names)):
        fig_roc.add_trace(go.Scatter(x=fpr[i], y=tpr[i], mode='lines', name=f'{class_names[i]} (AUC = {roc_auc[i]:0.2f})'))
    fig_roc.update_layout(title='ROC Curve',
                          xaxis_title='False Positive Rate',
                          yaxis_title='True Positive Rate',
                          width=750,
                          height=750)
    if save_path:
        fig_roc.write_html(save_path)  # Save the plot to the specified path as HTML
    fig_roc.show()

    print(f'Accuracy: {accuracy}')
    print(f'Top-5 Accuracy: {top5_accuracy}')
    print(f'F1 Score: {f1}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')

# Predict labels and generate confusion matrix
def plot_confusion_matrix_and_metrics(generator, model, class_names, title, save_path=None):
    # Get true labels and predicted labels
    true_labels, predicted_labels, predicted_probabilities = get_true_and_predicted(generator, model)
    print(f"True labels: {true_labels}")
    print(f"Predicted labels: {predicted_labels}")
    # Generate confusion matrix
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    plot_confusion_matrix(conf_matrix, class_names, title, save_path)

    # Calculate metrics and plot ROC-AUC curve
    calculate_metrics_and_plot_roc(np.array(true_labels), np.array(predicted_probabilities), class_names, title, save_path=None)

# Function to calculate top-k accuracy
def top_k_accuracy(true_labels, predicted_probabilities, k=5):
    top_k_predictions = np.argsort(predicted_probabilities, axis=1)[:, -k:]  # Get top k predictions
    matches = np.zeros_like(true_labels)
    for i in range(k):
        matches += (top_k_predictions[:, i] == true_labels)
    top_k_accuracy = np.mean(matches)
    return top_k_accuracy

## Model loading and evaluation on test

In [ ]:
TEST_DATA_DIRECTORY = "/kaggle/input/sneakers-dataset-test-split/TestDataset"

TEST_CLASS_MAPPING = {
  '10010': 0,
  '10011': 1,
  '10024': 2,
  '10050': 3,
  '10053': 4,
  '10031': 5,
  '10067': 6,
  '10068': 7,
  '10003': 8,
  '10004': 9,
  '10013': 10,
  '10014': 11,
  '10015': 12,
  '10016': 13,
  '10017': 14,
  '10018': 15,
  '10023': 16,
  '10046': 17,
  '10047': 18,
  '10001': 19,
  '10002': 20,
  '10005': 21,
  '10006': 22,
  '10008': 23,
  '10019': 24,
  '10020': 25,
  '10039': 26,
  '10041': 27,
  '10077': 28,
  '10091': 29,
  '10092': 30,
  '10104': 31,
  '10105': 32
}


# Create train and validation data generators
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

test_generator = test_datagen.flow_from_directory(
    TEST_DATA_DIRECTORY,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',  # specify this as training data
    seed=SEED,
    classes=TEST_CLASS_MAPPING
)


all_labels = set(test_generator.labels)
number_of_labels = len(all_labels)

In [ ]:
all_labels

In [ ]:
test_generator.class_indices

In [ ]:
reversed_test_mapping = {v: k for k, v in test_generator.class_indices.items()}
reversed_test_mapping

In [ ]:
test_class_names = [reversed_test_mapping[test_label] for test_label in all_labels]
test_class_names

In [ ]:
def plot_test_confusion_matrix(conf_matrix, class_names, title, save_path=None):
    
    trace = go.Heatmap(z=np.flipud(conf_matrix),  # Reverse the y-axis
                       x=class_names,
                       y=class_names[::-1],  # Reverse the y-axis labels
                       colorscale='Blues',
                       colorbar=dict(title='Count'),
                       )
    layout = go.Layout(title=title,
                       xaxis=dict(title='Predicted labels'),
                       yaxis=dict(title='True labels'),
                       width=500,  # Specify width of the plot
                       height=500,  # Specify height of the plot
#                        legend=dict(title='Legend'),  # Include legend
                       )
    fig = go.Figure(data=[trace], layout=layout)
    if save_path:
        fig.write_html(save_path)  # Save the plot to the specified path as HTML
    fig.show()
    
# Function to calculate metrics and plot ROC-AUC curve
def calculate_test_metrics_and_plot_roc(true_labels, predicted_probabilities, class_names, title, save_path=None):
    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predicted_probabilities.argmax(axis=1))

    # Calculate top-5 accuracy
    top5_accuracy = top_k_accuracy(true_labels, predicted_probabilities, k=5)

    # Calculate F1 score
    f1 = f1_score(true_labels, predicted_probabilities.argmax(axis=1), average='macro')

    # Calculate precision
    precision = precision_score(true_labels, predicted_probabilities.argmax(axis=1), average='macro')

    # Calculate recall
    recall = recall_score(true_labels, predicted_probabilities.argmax(axis=1), average='macro')

    # Plot ROC-AUC curve
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i, v in enumerate(class_names):
        print(i, v)
        i = test_generator.class_indices[v]
        fpr[i], tpr[i], _ = roc_curve(true_labels == v, predicted_probabilities[:, i])
#         print(predicted_probabilities[:, i])
#         print(predicted_probabilities[:, test_generator.class_indices[v]])
        print(fpr[i], tpr[i])
        roc_auc[i] = auc(fpr[i], tpr[i])
#         print(roc_auc[i])

    # Plot ROC-AUC curve
    fig_roc = go.Figure()
    for i, v in enumerate(class_names):
        i = test_generator.class_indices[v]
        fig_roc.add_trace(go.Scatter(x=fpr[i], y=tpr[i], mode='lines', name=f'{v} (AUC = {roc_auc[i]:0.2f})'))
    fig_roc.update_layout(title='ROC Curve',
                          xaxis_title='False Positive Rate',
                          yaxis_title='True Positive Rate',
                          width=500,
                          height=500)
    if save_path:
        fig_roc.write_html(save_path)  # Save the plot to the specified path as HTML
    fig_roc.show()

    print(f'Accuracy: {accuracy}')
    print(f'Top-5 Accuracy: {top5_accuracy}')
    print(f'F1 Score: {f1}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')

In [ ]:
# Get true labels and predicted labels
test_true_labels, test_predicted_labels, test_predicted_probabilities = get_true_and_predicted(test_generator, model)

# Generate confusion matrix
conf_matrix = confusion_matrix(test_true_labels, test_predicted_labels)

plot_test_confusion_matrix(conf_matrix, ["10067"] + test_class_names,  "Confusion Matrix - Test Set", save_path=f"{MODEL_SAVING_PATH}/test_set_plotly.html")

In [ ]:
# # Calculate metrics and plot ROC-AUC curve
# calculate_test_metrics_and_plot_roc(np.array(test_true_labels), np.array(test_predicted_probabilities), ["10067"] + test_class_names, "Test Set Roc Auc", save_path=f"{MODEL_SAVING_PATH}/test_set_roc_auc.html")

In [ ]:
test_generator.class_indices.get('1')

Check workability of code that loads model weights

In [ ]:
import shutil
from keras.models import load_model

LOAD_MODEL_PATH = "/kaggle/working/models/DATA_V2_<function EfficientNetB1 at 0x7bbeba678d30>_0Frozen_32Batch_240x240Size_AugV2/model.keras"
LOAD_MODEL_PATH_TO_USE = "/kaggle/working/copied_model.keras"

# Copy the model to the destination
shutil.copy(LOAD_MODEL_PATH, LOAD_MODEL_PATH_TO_USE)

# Load the model from the destination path
# downloaded_model = load_model(LOAD_MODEL_PATH_TO_USE)

# Below loading is incorrect

In [ ]:
# number_of_labels = 33

# def create_model():
#     base_model = MODEL_CLASS_CALLBACK(input_tensor=input_layer, weights='imagenet', include_top=False)
#     # # Freeze layers
#     for layer in base_model.layers[:NUMBER_LAYERS_TO_FREEZE]:
#         layer.trainable = False
    
#     x = base_model.output
#     x = GlobalAveragePooling2D()(x)
#     predictions = Dense(number_of_labels, activation='softmax')(x)
#     model = Model(inputs=base_model.input, outputs=predictions)
#     return model

# downloaded_model = create_model()

# # Compile the model
# downloaded_model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=["categorical_accuracy"])# metrics=["sparse_categorical_accuracy"])
# # downloaded_model = 
# downloaded_model.load_weights("/kaggle/working/models/DATA_V2_<function EfficientNetB1 at 0x7f46f2774d30>_0Frozen_32Batch_240x240Size_AugV2/model_weights_epoch_13.weights.h5")

# # Load the model from the destination path
# # downloaded_model = load_model(LOAD_MODEL_PATH_TO_USE)

In [ ]:
# downloaded_model =  tf.keras.models.load_model(LOAD_MODEL_PATH)
# downloaded_model.summary()

In [ ]:
# # Get true labels and predicted labels
# test_true_labels, test_predicted_labels, test_predicted_probabilities = get_true_and_predicted(test_generator, downloaded_model)

# # Generate confusion matrix
# conf_matrix = confusion_matrix(test_true_labels, test_predicted_labels)

# plot_test_confusion_matrix(conf_matrix, ["10067"] + test_class_names,  "Confusion Matrix - Test Set", save_path=f"{MODEL_SAVING_PATH}/test_set_plotly.html")

In [ ]:
# def create_model_test():
#     base_model = MODEL_CLASS_CALLBACK(input_tensor=input_layer, weights='imagenet', include_top=False)
#     # # Freeze layers
#     for layer in base_model.layers[:NUMBER_LAYERS_TO_FREEZE]:
#         layer.trainable = False
    
#     x = base_model.output
#     x = GlobalAveragePooling2D()(x)
#     predictions = Dense(33, activation='softmax')(x)
#     model = Model(inputs=base_model.input, outputs=predictions)
#     return model

# model_from_checkpoint = create_model_test()
# checkpoint_path = "/kaggle/working/models/DATA_V2_<function EfficientNetB1 at 0x7bbeba678d30>_0Frozen_32Batch_240x240Size_AugV2/model_weights_epoch_13.weights.h5"
# model_from_checkpoint.load_weights(checkpoint_path)

In [ ]:
# # Get true labels and predicted labels
# test_true_labels, test_predicted_labels, test_predicted_probabilities = get_true_and_predicted(test_generator, model_from_checkpoint)

# # Generate confusion matrix
# conf_matrix = confusion_matrix(test_true_labels, test_predicted_labels)

# plot_test_confusion_matrix(conf_matrix, ["10067"] + test_class_names,  "Confusion Matrix - Test Set", save_path=f"{MODEL_SAVING_PATH}/test_set_plotly.html")

In [ ]:
# model.save('my_model.h5')

In [ ]:
new_model = tf.keras.models.load_model('my_model.h5')
# new_model.summary()

In [ ]:
# Get true labels and predicted labels
test_true_labels, test_predicted_labels, test_predicted_probabilities = get_true_and_predicted(test_generator, new_model)

# Generate confusion matrix
conf_matrix = confusion_matrix(test_true_labels, test_predicted_labels)

plot_test_confusion_matrix(conf_matrix, ["10067"] + test_class_names,  "Confusion Matrix - Test Set", save_path=f"{MODEL_SAVING_PATH}/test_set_plotly.html")

## Conclusion



We loaded weights with `model.save("model.keras")` and read it with `tf.keras.models.load_model` and evaluation made it clear that model weights has not been loaded.


After reading a couple of articles we decided to save model in `.h5` format and it worked just fine.

Kudos to:

https://www.kaggle.com/code/vikramtiwari/tf2-tutorials-keras-save-and-restore-models